In [8]:
import sys
import os

notebook_dir = os.getcwd()  # Get current working directory (notebooks folder)
project_dir = os.path.abspath(os.path.join(notebook_dir, '..'))  # Go up one level
src_dir = os.path.join(project_dir, 'src')  # Path to scripts
scripts_dir = os.path.join(project_dir, 'scripts')
sys.path.append(src_dir)
sys.path.append(scripts_dir)


In [9]:
from load_data import DataLoader
from clean_data import DataCleaner
from visualize_data import Visualizer
from load_data import DataLoader
from clean_data import DataCleaner
from visualize_data import Visualizer
from correlation_analyzer import CorrelationAnalyzer
from financial_analyzer import FinancialAnalyzer


ImportError: cannot import name 'Analyzer' from 'pynance' (c:\Users\hp\Documents\10\nova-financial-solutions-week1\.venv\Lib\site-packages\pynance\__init__.py)

In [10]:
STOCK_FILES_INFO  = {
    "AAPL": "../data/raw/AAPL_historical_data.csv",
    "AMZN": "../data/raw/AMZN_historical_data.csv",
    "GOOG": "../data/raw/GOOG_historical_data.csv",
    "META": "../data/raw/META_historical_data.csv",
    "MSFT": "../data/raw/MSFT_historical_data.csv",
    "NVDA": "../data/raw/NVDA_historical_data.csv",
    "TSLA": "../data/raw/TSLA_historical_data.csv"
}


In [14]:
# --- 1. Load and Prepare News Data ---
print("Loading and preparing news data...")
news_loader = DataLoader(STOCK_FILES_INFO)
news_df = news_loader.load_data()

Loading and preparing news data...


TypeError: _path_isfile: path should be string, bytes, os.PathLike or integer, not DataLoader

In [15]:

news_with_sentiment = None
if news_df is not None:
    news_cleaner = DataCleaner(news_df)
    cleaned_news_df = news_cleaner.clean_news_data()

    news_text_analyzer = TextAnalyzer(cleaned_news_df)
    news_with_sentiment = news_text_analyzer.perform_sentiment_analysis()
    
    # Ensure date is normalized to day for merging
    news_with_sentiment['date'] = news_with_sentiment['date'].dt.normalize()
    print("\nNews data with sentiment prepared.")
    print(news_with_sentiment.head())
else:
    print("Failed to load news data. Cannot proceed with correlation analysis.")


# --- 2. Load and Prepare Stock Data for all tickers ---
print("\nLoading and preparing stock data for all tickers...")
stock_loader = DataLoader() # Initialize without news_file_path if only loading stock data
all_stock_dfs = stock_loader.load_local_stock_data(STOCK_FILES_INFO)

processed_stock_dfs = {}
if all_stock_dfs:
    for ticker, stock_df in all_stock_dfs.items():
        print(f"\nProcessing {ticker} stock data for correlation...")
        stock_cleaner = DataCleaner(stock_df)
        cleaned_stock_df = stock_cleaner.clean_stock_data()
        
        # Calculate daily returns
        financial_analyzer = FinancialAnalyzer(cleaned_stock_df)
        stock_df_with_returns = financial_analyzer.calculate_daily_returns()
        
        # Ensure stock_df_with_returns index is normalized to day
        stock_df_with_returns.index = stock_df_with_returns.index.normalize()
        
        processed_stock_dfs[ticker] = stock_df_with_returns
        print(f"Finished preparing {ticker} stock data with daily returns.")
else:
    print("No stock data was loaded. Cannot proceed with correlation analysis.")


# --- 3. Perform Correlation Analysis for all stocks ---
if news_with_sentiment is not None and processed_stock_dfs:
    print("\nStarting correlation analysis between news sentiment and stock returns for all loaded stocks...")
    correlation_analyzer = CorrelationAnalyzer(news_with_sentiment)
    all_correlations = correlation_analyzer.analyze_multiple_stocks(processed_stock_dfs)

    print("\n--- Correlation Results ---")
    for ticker, correlation in all_correlations.items():
        print(f"Correlation for {ticker}: {correlation:.4f}")

    # --- 4. Visualize Correlation (example for one stock) ---
    print("\nVisualizing correlation (example for the first successfully processed stock)...")
    if all_correlations:
        first_ticker = next(iter(all_correlations)) # Get the first ticker
        first_stock_df = processed_stock_dfs[first_ticker]

        # Re-merge for plotting, as correlation_analyzer.align_and_merge_data is for one stock at a time
        temp_correlation_analyzer = CorrelationAnalyzer(news_with_sentiment)
        df_for_plot = temp_correlation_analyzer.align_and_merge_data(first_stock_df)

        if df_for_plot is not None and not df_for_plot.empty:
            visualizer = Visualizer()
            visualizer.plot_sentiment_vs_returns_scatter(df_for_plot, title=f"Avg. Daily Sentiment vs. Daily Returns for {first_ticker}")
            visualizer.plot_correlation_heatmap(df_for_plot, ['Avg_Daily_Sentiment', 'Daily_Return'], title=f"Correlation Heatmap for {first_ticker}")
        else:
            print("Could not generate plots for correlation (merged data is empty).")
    else:
        print("No correlations calculated to plot.")
else:
    print("Missing news data or stock data to perform correlation analysis.")


NameError: name 'news_df' is not defined